TASK 1

In [17]:
from bs4 import BeautifulSoup
import os

# Check if the weather.html file exists
if os.path.exists('weather.html'):
    with open('weather.html', 'r', encoding= 'utf-8') as file:
        html_content = file.read()
    soup = BeautifulSoup(html_content, 'html.parser')

    forecast = []
    tablerows = soup.find_all('tr')[1:]
    for row in tablerows:
        cols = row.find_all('td')
        day = cols[0].text
        temp = int(cols[1].text.strip().replace('°C', ''))
        condition = cols[2].text
        forecast.append((day,temp,condition))
    print("Weather Forecast: ")
    for day,temp,condition in forecast:
        print(f"{day}: {temp}°C, {condition}")
    hight_temp = max(forecast, key=lambda x: x[1])
    print(f"Highest Temperature: {hight_temp[0]} ({hight_temp[1]}°C)")
    sunny_days = [day for day, temp, condition in forecast if condition == 'Sunny']
    print(f"Days with 'Sunny' condition: {', '.join(sunny_days)}")

    average_temp = sum(temp for day, temp, condition in forecast)/len(forecast)
    print(f"Average temp for the week : {average_temp:.2f}°C")
else:
    print("weather.html file not found.")


Weather Forecast: 
Monday: 25°C, Sunny
Tuesday: 22°C, Cloudy
Wednesday: 18°C, Rainy
Thursday: 20°C, Partly Cloudy
Friday: 30°C, Sunny
Highest Temperature: Friday (30°C)
Days with 'Sunny' condition: Monday, Friday
Average temp for the week : 23.00°C


TASK 2

In [5]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import csv
import time

# Connect to SQLite database and create the table if it doesn't exist
with sqlite3.connect("jobs.db") as con:
    cursor = con.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS jobs (
            Job_Title TEXT,
            Company_Name TEXT,
            Location TEXT,
            Job_Description TEXT,
            Application_Link TEXT,
            PRIMARY KEY (Job_Title, Company_Name, Location)
        )
    ''')

def scrape_jobs():
    url = "https://realpython.github.io/fake-jobs"
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        return []

    soup = BeautifulSoup(response.content, 'html.parser')
    job_listing = []

    for job_card in soup.find_all('div', class_='card-content'):
        job_title = job_card.find('h2', class_='title is-5').text.strip() if job_card.find('h2', class_='title is-5') else 'No title'
        company_name = job_card.find('h3', class_='subtitle is-6 company').text.strip() if job_card.find('h3', class_='subtitle is-6 company') else 'No company name'
        location = job_card.find('p', class_='location').text.strip() if job_card.find('p', class_='location') else 'No location'
        application_link = job_card.find('a', string='Apply').get('href').strip() if job_card.find('a', string='Apply') else None

        if application_link:
            response1 = requests.get(application_link)
            if response1.status_code != 200:
                print(f"Failed to fetch job description for {job_title}")
                continue

            soup1 = BeautifulSoup(response1.content, 'html.parser')
            job_description = soup1.find('div', class_='content').find('p').text.strip() if soup1.find('div', class_='content') else 'No description'
            job_listing.append((job_title, company_name, location, job_description, application_link))
        
        # Delay between requests to avoid overloading the server
        time.sleep(1)

    return job_listing

def incremental_load(job_listing):
    for job in job_listing:
        cursor.execute('''
            INSERT OR REPLACE INTO jobs (Job_Title, Company_Name, Location, Job_Description, Application_Link)
            VALUES (?, ?, ?, ?, ?)
        ''', job)
        con.commit()

def filter_jobs(location=None, company_name=None):
    query = "SELECT * FROM jobs WHERE 1=1"
    params = []

    if location:
        query += " AND Location LIKE ?"
        params.append(f"%{location}%")
    
    if company_name:
        query += " AND Company_Name LIKE ?"
        params.append(f"%{company_name}%")

    cursor.execute(query, params)
    return cursor.fetchall()

def export_to_csv(filtered_jobs, filename="filtered_jobs.csv"):
    with open(filename, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["Job Title", "Company Name", "Location", "Job Description", "Application Link"])
        writer.writerows(filtered_jobs)
    print(f"Filtered jobs have been exported to {filename}.")

def main():
    job_listing = scrape_jobs()
    if job_listing:
        incremental_load(job_listing)
        print("New job listings have been added or updated in the database.")

    # Example: Filter jobs by location or company name
    location_filter = "Stewartbury, AA"  # Example location
    company_filter = "Payne, Roberts and Davis"  # Example company name

    filtered_jobs = filter_jobs(location=location_filter, company_name=company_filter)
    print(f"Filtered jobs (Location: {location_filter}, Company: {company_filter}):")
    for job in filtered_jobs:
        print(job)

    # Export filtered results to CSV
    export_to_csv(filtered_jobs)

if __name__ == "__main__":
    main()


New job listings have been added or updated in the database.
Filtered jobs (Location: Stewartbury, AA, Company: Payne, Roberts and Davis):
('Senior Python Developer', 'Payne, Roberts and Davis', 'Stewartbury, AA', 'Professional asset web application environmentally friendly detail-oriented asset. Coordinate educational dashboard agile employ growth opportunity. Company programs CSS explore role. Html educational grit web application. Oversea SCRUM talented support. Web Application fast-growing communities inclusive programs job CSS. Css discussions growth opportunity explore open-minded oversee. Css Python environmentally friendly collaborate inclusive role. Django no experience oversee dashboard environmentally friendly willing to learn programs. Programs open-minded programs asset.', 'https://realpython.github.io/fake-jobs/jobs/senior-python-developer-0.html')
Filtered jobs have been exported to filtered_jobs.csv.


TASK 3

In [19]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import json
import time
# Brauzer driverini ishga tushirish
driver = webdriver.Chrome()

try:
    # Saytga o'tish
    driver.get("https://www.demoblaze.com/")
    
    # Laptops tugmasini topish va bosish
    buttons = driver.find_elements(By.ID, "itemc")
    for button in buttons:
        if "byCat('notebook')" in button.get_attribute('onclick'):
            button.click()
            print("Visiting laptop page...")
            break

    # Sahifaning to'liq yuklanishini kutish
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "next2"))
    )

    # Next tugmasini bosish
    next_button = driver.find_element(By.ID, "next2")
    next_button.click()
    print("Next page button clicked...")

    time.sleep(10)
    laptops_data = []
    products = driver.find_elements(By.CLASS_NAME, 'col-lg-4.col-md-6.mb-4')
    for product in products:
        name = product.find_element(By.CLASS_NAME, 'card-title').text
        price = product.find_element(By.TAG_NAME, 'h5').text
        description = product.find_element(By.CLASS_NAME, 'card-text').text

        laptops_data.append(
            {
            "name": name,
            "price": price,
            "description": description
            }
            )
    with open("laptops_data.json", "w", encoding="utf-8") as f:
        json.dump(laptops_data, f, indent=4)

finally:
    # Brauzerni yopish
    driver.quit()


Visiting laptop page...
Next page button clicked...
